In [ ]:
import aextl
import numpy
import os
import yaml
import math

import scipy
import scipy.stats

cfg = aextl.config_for_dir("/home/moritz/Seminar/experimental-algorithmics-for-na-paper/experiments")

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')

%matplotlib inline

# Read in Experimental Results

In [ ]:
# Load exact betweenness values for each instance
exactValues = dict()
exactValueDir = "/home/moritz/Seminar/output/brandes/"
for instance in cfg.all_instances():
    try:
        with open(os.path.join(exactValueDir, instance.filename + '.out-full'), 'r') as f:
            exactValues[instance.filename] = numpy.loadtxt(f)
    except e:
        print(e)

In [ ]:
# Load the overall running times into a single list.
def load_file(run, f):
    data = yaml.load(f)
    return {
        'experiment': run.experiment.name,
        'instance': run.instance.filename,
        'run_time': data['run_time'],
        'parameters' : data['parameters']
    }

runs = cfg.collect_successful_results(load_file)

In [ ]:
foundInstances = set([run['instance'] for run in runs if 'iters_per_step' in run['parameters'].keys()])
len(foundInstances)

In [ ]:
sortedInstances = sorted(foundInstances, key=lambda x : len(exactValues[x]) )
sizes = [len(exactValues[instance]) for instance in sortedInstances]
logSizes = [math.log(size) for size in sizes]
len(sortedInstances)

In [ ]:
classifiedTimes = dict()
for run in runs:
    if not 'iters_per_step' in run['parameters'].keys():
        continue
    if run['experiment'] != 'kadabra':
        raise ValueError("iters_per_step parameter exists, but algo is " + run['experiment'] + ".")
    iters = run['parameters']['iters_per_step']
    instance = run['instance']
    if not iters in classifiedTimes.keys():
        classifiedTimes[iters] = dict()
    if not instance in classifiedTimes[iters].keys():
        classifiedTimes[iters][instance] = list()
    
    classifiedTimes[iters][instance].append(run)

In [ ]:
sortedTimes = dict()
for ips in classifiedTimes.keys():
    sortedTimes[ips] = list()
    for instance in sortedInstances:
        sortedTimes[ips].append(classifiedTimes[ips][instance][0]['run_time'])

In [ ]:
for ips in classifiedTimes.keys():
    plt.plot(sizes, sortedTimes[ips], label=str(ips))
plt.xscale('log')
plt.yscale('log')
plt.xlabel('nodes')
plt.ylabel('seconds')
plt.legend()
plt.show()

In [ ]:
import scipy
import scipy.stats

In [ ]:
import math
import pymc3 as pm